In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pyautogui

# Initialization of MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Indices for left eye landmarks
left_eye_indices = [33, 160, 158, 133, 153, 144]
# Indices for right eye landmarks
right_eye_indices = [362, 385, 387, 263, 373, 380]

# Function to calculate the distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to determine the state of the eyes
def are_eyes_closed(landmarks):
    # Calculate the distance between the upper and lower parts of the eyes
    left_eye_height = calculate_distance(landmarks[left_eye_indices[1]], landmarks[left_eye_indices[5]])
    right_eye_height = calculate_distance(landmarks[right_eye_indices[1]], landmarks[right_eye_indices[5]])
    
    # Calculate the horizontal distance for both eyes
    left_eye_width = calculate_distance(landmarks[left_eye_indices[0]], landmarks[left_eye_indices[3]])
    right_eye_width = calculate_distance(landmarks[right_eye_indices[0]], landmarks[right_eye_indices[3]])
    
    # Ratio of height to width
    left_eye_ratio = left_eye_height / left_eye_width
    right_eye_ratio = right_eye_height / right_eye_width
    
    # Threshold to determine closed eyes
    threshold = 0.2
    
    return left_eye_ratio < threshold or right_eye_ratio < threshold

# Initialization of variables for tracking blinks
blink_start_time = None
last_blink_time = None
min_duration_for_click = 0.5
blink_duration = 0.2

# Parameter for cursor movement
move_distance = 20  # Distance the cursor moves when eyes move
threshold_sensitivity = 0.2  # Sensitivity threshold for eye movement

# Video capture from the camera
cap = cv2.VideoCapture(0)

# Initial cursor coordinates
initial_cursor_x, initial_cursor_y = pyautogui.position()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the image with MediaPipe Face Mesh
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw the face mesh
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_tesselation_style())
            
            # Convert face landmarks to a numpy array
            landmarks = np.array([(lm.x * frame.shape[1], lm.y * frame.shape[0]) for lm in face_landmarks.landmark])
            
            # Draw the eyes
            for i in left_eye_indices + right_eye_indices:
                cv2.circle(frame, (int(landmarks[i][0]), int(landmarks[i][1])), 1, (0, 255, 0), -1)
            
            # Get the coordinates of the center of the face
            nose_tip = landmarks[1]  # Index 1 corresponds to the tip of the nose
            screen_width, screen_height = pyautogui.size()
            
            # Normalize the coordinates of the nose
            normalized_x = nose_tip[0] / frame.shape[1]
            normalized_y = nose_tip[1] / frame.shape[0]
            
            # Check eye movement along the x and y axes
            if abs(normalized_x - 0.5) > threshold_sensitivity:
                direction_x = -1 if normalized_x > 0.5 else 1
                initial_cursor_x += direction_x * move_distance
            
            if abs(normalized_y - 0.5) > threshold_sensitivity:
                direction_y = 1 if normalized_y > 0.5 else -1
                initial_cursor_y += direction_y * move_distance
            
            pyautogui.moveTo(initial_cursor_x, initial_cursor_y)
            
            # Determine the state of the eyes
            if are_eyes_closed(landmarks):
                if blink_start_time is None:
                    blink_start_time = time.time()
                cv2.putText(frame, "Eyes Closed", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                if blink_start_time is not None:
                    duration = time.time() - blink_start_time
                    if duration >= min_duration_for_click:
                        pyautogui.click()
                    elif last_blink_time is not None and time.time() - last_blink_time <= blink_duration:
                        pyautogui.doubleClick()
                    last_blink_time = time.time()
                    blink_start_time = None
                cv2.putText(frame, "Eyes Open", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    cv2.imshow('MediaPipe Face Mesh', frame)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
face_mesh.close()
